<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_agent_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
# Se instalan las librerías.
!pip install langchain==0.3.21
!pip install langchain-community==0.3.20
!pip install langchain_deepseek==0.1.2
!pip install langgraph==0.3.31
!pip install duckduckgo-search==8.0.0

In [ ]:
# Se importan las librerías.
import warnings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_deepseek import ChatDeepSeek
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
import pandas as pd

# Filtro para advertencias.
warnings.filterwarnings('ignore')

<br>

# Añadiendo y utilizando una herramienta de búsqueda web

Existen diferentes herramientas predefinidas en LangChain, que pueden ser coconsultadas en la siguiente URL: https://python.langchain.com/docs/integrations/tools/

Nosotros utilizaremos el buscador web DuckDuckGo, el cual puede ser consultado en: https://python.langchain.com/docs/integrations/tools/ddg/

In [ ]:
# Definimos la herramienta del buscador web.
search = DuckDuckGoSearchResults(max_results=5, description="""Herramienta que
puede ser utilizada para buscar información en internet, se puede buscar el
clima del día actual, por ejemplo: ¿cuál es la temperatura en Guadalajara
Jalisco el día de hoy?""")

# Se prueba.
search.invoke("Temperatura de hoy en Guadalajara Jalisco")

Ahora utilizaremos este buscador como herramienta de una LLM.

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

# Se define el modelo a utilizar y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=200
      )

In [ ]:
# Se añade el buscador web a una lista de herramientas.
tools = [search]

# Se añade al modelo.
model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="¿Cuál es la temperatura en Guadalajara, Jalisco, México?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response

# Creando agente con LangGraph

Ahora utilizaremos `create_react_agent` para crear un agente que utilice esta herramienta. Este flujo funciona como un grafo, donde el nodo "agent" llama al modelo de lenguaje con la lista de mensajes añadidos. Si el AIMessage resultante contiene tool_calls, el grafo llamará entonces a los "tools". El nodo "tools" ejecuta las herramientas (una herramienta por cada tool_call) y añade las respuestas a la lista de mensajes como objetos ToolMessage. Luego, el nodo del agente vuelve a llamar al modelo de lenguaje. Este proceso se repite hasta que la respuesta ya no contenga más tool_calls. Finalmente, el agente devuelve la lista completa de mensajes como un diccionario que contiene la clave "messages".

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/04/graph_lc.png" width="200" /></center>

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/04/graph_lc_2.png" width="800" /></center>

In [ ]:
# Se crea el agente pasando el modelo y las herramientas como argumentos.
agent_executor = create_react_agent(model, tools)

# Se realiza una prueba.
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Cuál es la temperatura en Guadalajara, Jalisco, México?")]}
)
response["messages"]

In [ ]:
# Se filtra el mensaje de tipo AIMessage (aunque usualmente es el último)
ai_messages = [m for m in response["messages"] if isinstance(m, AIMessage)]
print(ai_messages[-1].content)

<br>

# Creando un agente multi-herramienta (multi-tool agent)

Para crear un agente multi-herramienta añadiremos una nueva herramienta a la lista y recrearemos el agente.

Lo haremos para crear un agente que atienda a una tienda electrónica de accesorios para mascotas.

El agente debe hacer lo siguiente:



*   Investigar la temperatura de la ciudad donde vive el cliente.
*   Conociendo la temperatura, y el tipo de mascota del cliente (perro o gato), extraer información de los productos que puede ofrecer de un CSV.
* Dar la información al cliente.

<br>
<center><img src="https://dialektico.com/wp-content/uploads/2025/04/Diagrama_agente.drawio.png" width="1000" /></center>


## Se crea la herramienta de búsqueda en CSV

In [ ]:
def obtener_prendas_para_mascota(ruta_csv: str, animal: str, temperatura_aprox: float) -> list[str]:
    """
    Filtra una tabla CSV de prendas para mascotas y devuelve aquellas adecuadas
    según el tipo de animal (perro o gato) y la temperatura actual.

    La función clasifica la temperatura en una de tres categorías:
        - 'frío': temperatura menor a 15°C
        - 'templado': temperatura entre 15°C y 25°C (inclusive)
        - 'caluroso': temperatura mayor a 25°C

    Luego, filtra las prendas que coincidan con la clasificación de temperatura y el animal especificado.

    Args:
        ruta_csv (str): Ruta al archivo CSV que contiene la tabla de prendas.
        animal (str): Tipo de mascota, por ejemplo, 'gato' o 'perro'.
        temperatura_actual (float): Temperatura en grados Celsius.

    Returns:
        list[str]: Lista de nombres de prendas recomendadas para el animal y clima dados.

    Raises:
        FileNotFoundError: Si no se encuentra el archivo CSV en la ruta especificada.
        ValueError: Si el tipo de animal no está presente en la tabla.

    Example:
        >>> obtener_prendas_para_mascota('prendas_mascotas.csv', 'perro', 10)
        ['chamarra acolchada impermeable', 'suéter térmico de felpa']
    """
    # Cargar la tabla
    df = pd.read_csv(ruta_csv)

    # Clasificar la temperatura
    if temperatura_aprox < 15:
        clasificacion_temp = 'frío'
    elif 15 <= temperatura_aprox <= 25:
        clasificacion_temp = 'templado'
    else:
        clasificacion_temp = 'caluroso'

    # Verificar si el animal está en el dataset
    if animal.lower() not in df['animal'].str.lower().unique():
        raise ValueError(f"Animal '{animal}' no encontrado en la tabla.")

    # Filtrar según clasificación de temperatura y animal
    prendas_filtradas = df[
        (df['temperatura'] == clasificacion_temp) &
        (df['animal'].str.lower() == animal.lower())
    ]

    return prendas_filtradas['nombre_prenda'].tolist()

In [ ]:
# Se prueba la función.
obtener_prendas_para_mascota('/content/productos.csv', 'perro', 28)

In [ ]:
# Se crea el agente con herramientas nuevas.
tools = [search, obtener_prendas_para_mascota]
agent_executor = create_react_agent(model, tools)

# Se añaden instrucciones.
system_message = """Eres un asistente de una tienda en línea que vende productos
        para mascotas. El cliente te dirá qué mascota tiene (perro o gato) y en
        qué ciudad vivo, y tú debes de seguir los siguientes pasos:
        - Debes de utilizar la herramienta search para obtener la temperatura
        aproximada de la ciudad donde vive, colocando en el buscador:
        cuál es el clima actual en {ciudad del cliente}.
        - Utilizando la información del buscador, utilizarás la herramienta
        obtener_prendas_para_mascota, añadiendo los argumentos correspondientes,
        donde el csv con la información está en /content/productos.csv, y añades
        tipo de animal y temperatura aproximada para obtener los datos.
        - Utilizando estos datos, dile al cliente qué prendas puedes ofrecer.
        """


In [ ]:
# Se utiliza el agente haciendo stream en la respuesta.
for step, metadata in agent_executor.stream(
    {"messages": [SystemMessage(
        content=system_message),
                  HumanMessage(content="Tengo un perro y vivo en Medellín, Colombia")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")

Más información en:


*   https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent
*   https://langchain-ai.github.io/langgraph/reference/types/#langgraph.types.StreamMode



<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License